# 🏗️ Tutorial 1: Real IFC Data Extraction - Vilamalla Industrial Complex

**What you'll learn:**
- How to extract building data from real IFC files
- Work with the Vilamalla Industrial Complex project
- Visualize building elements (rooms, doors, walls)
- Analyze building data with pandas

This tutorial uses **real architectural data** from the Vilamalla Industrial Complex, showing how our IFC extraction system works with actual building information models.

## 📁 What is an IFC File?

**IFC (Industry Foundation Classes)** is the international standard for Building Information Modeling (BIM):

- 📐 **Contains**: 3D geometry, building elements, relationships, properties
- 🏢 **Elements**: Walls, doors, rooms, levels, structural elements
- 🔗 **Relationships**: How elements connect and relate to each other
- 📊 **Metadata**: Materials, dimensions, performance characteristics

Our IFC file: `VILAMALLA_ARQ_V6_TALLER_arq_20251032.ifc` (6.5 MB)


## 🚀 Step 1: Load the Real Building Data

First, let's load our pre-extracted building data from the Vilamalla Industrial Complex:

In [ ]:
# Import our custom building data loader
import sys
from pathlib import Path
sys.path.append('..')

from scripts.load_building_data import load_vilamalla_building, quick_building_overview
import matplotlib.pyplot as plt
import pandas as pd

# Load the real building data
print("🏗️ Loading Vilamalla Industrial Complex...")
loader = load_vilamalla_building()

# Show overview
quick_building_overview(loader)

## 🏢 Step 2: Explore Building Levels

Let's examine the different levels in this industrial complex:

In [ ]:
print("📋 Building Levels in Detail:\n")

for i, level in enumerate(loader.levels, 1):
    level_name = level['name']
    elevation = level['elevation']
    rooms = level.get('rooms', [])
    doors = level.get('doors', [])
    walls = level.get('walls', [])
    
    print(f"{i:2}. Level: {level_name}")
    print(f"     Elevation: {elevation:.2f} m")
    print(f"     Elements: {len(rooms)} rooms, {len(doors)} doors, {len(walls)} walls")
    
    if rooms:
        total_area = sum(room['area'] for room in rooms)
        print(f"     Total area: {total_area:.0f} m²")
    print()

## 🚪 Step 3: Analyze Doors

Let's examine the doors in this building and their compliance with building codes:

In [ ]:
from scripts.load_building_data import analyze_door_compliance

print("🚪 Door Analysis:\n")
print(f"Total doors found: {len(loader.all_doors)}")

# Show first few doors as examples
print("\n📋 Sample doors:")
for door in loader.all_doors[:5]:
    level_name = "Unknown"
    for level in loader.levels:
        if door in level.get('doors', []):
            level_name = level['name']
            break
    
    print(f"  {door['id']}: {door['width_mm']}mm x {door['height_mm']}mm")
    print(f"    Level: {level_name[:20]}...")
    print(f"    Type: {door['door_type']}")
    print()

# Compliance analysis
compliance = analyze_door_compliance(loader)
print(f"\n✅ Compliance Analysis:")
print(f"   Compliance rate: {compliance['compliance_rate']:.1f}%")
print(f"   Compliant doors: {compliance['compliant_count']}/{compliance['total_doors']}")

if compliance['non_compliant_doors']:
    print(f"\n❌ Non-compliant doors:")
    for door in compliance['non_compliant_doors']:
        print(f"   {door['id']}: {door['width_mm']}mm (needs {door['deficit_mm']}mm more)")
else:
    print("\n🎉 All doors meet minimum width requirements!")

## 🧱 Step 4: Examine Walls

Let's analyze the wall structure of the building:

In [ ]:
print("🧱 Wall Analysis:\n")
print(f"Total walls: {len(loader.all_walls)}")

# Analyze wall materials
materials = {}
thicknesses = []
heights = []

for wall in loader.all_walls:
    material = wall.get('material', 'unknown')
    materials[material] = materials.get(material, 0) + 1
    thicknesses.append(wall['thickness_mm'])
    heights.append(wall['height_mm'])

print("📊 Wall materials:")
for material, count in materials.items():
    print(f"   {material}: {count} walls ({count/len(loader.all_walls)*100:.1f}%)")

print(f"\n📏 Wall dimensions:")
print(f"   Average thickness: {sum(thicknesses)/len(thicknesses):.0f} mm")
print(f"   Average height: {sum(heights)/len(heights):.0f} mm")
print(f"   Thickness range: {min(thicknesses):.0f} - {max(thicknesses):.0f} mm")
print(f"   Height range: {min(heights):.0f} - {max(heights):.0f} mm")

## 📊 Step 5: Create Data Visualizations

Now let's create some visualizations to better understand our building:

In [ ]:
# Create comprehensive statistics dashboard
print("📈 Creating building statistics dashboard...")
loader.create_statistics_dashboard()
plt.show()

## 🗺️ Step 6: Visualize Building Levels

Let's visualize the level with the most elements (doors and walls):

In [ ]:
# Find the level with most doors (most activity)
level_with_most_doors = max(loader.levels, key=lambda x: len(x.get('doors', [])))
level_name = level_with_most_doors['name']

print(f"🗺️ Visualizing level: {level_name}")
print(f"   Doors: {len(level_with_most_doors.get('doors', []))}")
print(f"   Walls: {len(level_with_most_doors.get('walls', []))}")
print()

# Create visualization
fig = loader.visualize_level(level_name, figsize=(14, 10))
plt.show()

## 📊 Step 7: Export to Pandas for Advanced Analysis

Let's export our data to pandas DataFrames for more detailed analysis:

In [ ]:
# Export to DataFrames
dataframes = loader.export_to_dataframes()

rooms_df = dataframes['rooms']
doors_df = dataframes['doors']
walls_df = dataframes['walls']

print("📊 Building Data as Pandas DataFrames:")
print(f"\n🏢 Rooms DataFrame ({len(rooms_df)} rows):")
print(rooms_df.head())

print(f"\n🚪 Doors DataFrame ({len(doors_df)} rows):")
print(doors_df[['id', 'width_mm', 'height_mm', 'door_type', 'is_emergency_exit']].head())

print(f"\n🧱 Walls DataFrame ({len(walls_df)} rows):")
print(walls_df[['id', 'thickness_mm', 'height_mm', 'material']].head())

## 🔍 Step 8: Advanced Analysis with Pandas

Now let's do some advanced analysis using pandas:

In [ ]:
# Analyze room distribution by level
print("🏢 Room Distribution by Level:")
room_stats = rooms_df.groupby('level').agg({
    'area_m2': ['count', 'sum', 'mean'],
    'occupancy_load': 'sum'
}).round(1)
print(room_stats)

print("\n🚪 Door Statistics:")
door_stats = doors_df.describe()[['width_mm', 'height_mm']].round(1)
print(door_stats)

print("\n🧱 Wall Material Analysis:")
wall_materials = walls_df['material'].value_counts()
print(wall_materials)

# Find potential issues
print("\n⚠️  Potential Issues:")
narrow_doors = doors_df[doors_df['width_mm'] < 800]
if len(narrow_doors) > 0:
    print(f"   - {len(narrow_doors)} doors below 800mm width (emergency requirement)")
else:
    print("   - All doors meet 800mm emergency width requirement ✅")

thin_walls = walls_df[walls_df['thickness_mm'] < 100]
if len(thin_walls) > 0:
    print(f"   - {len(thin_walls)} walls below 100mm thickness (potentially structural concern)")
else:
    print("   - All walls have adequate thickness ✅")

## 🎯 Summary

In this tutorial, you learned:

1. ✅ **IFC Files** contain rich building information including geometry and relationships
2. ✅ **Real Data Extraction** from the Vilamalla Industrial Complex (9 levels, 23 doors, 102 walls)
3. ✅ **Data Analysis** using custom loader and pandas for compliance checking
4. ✅ **Visualization** of building elements and statistics
5. ✅ **Advanced Analytics** to identify potential design issues

**Key Findings from Vilamalla Complex:**
- 🏢 9 building levels with varying functions
- 🚪 23 doors, all compliant with width requirements
- 🧱 102 walls, primarily concrete construction
- 📐 720 m² total area across all levels

**Next**: Tutorial 2 - Geometric Calculations & Route Analysis using this real data

## 💡 Try It Yourself

Experiment with the code above:

1. **Filter data**: Try `doors_df[doors_df['door_type'] == 'single']` to see only single doors
2. **Visualize different levels**: Change `level_name` in the visualization code
3. **Custom analysis**: Create your own queries using pandas
4. **Export data**: Save DataFrames to CSV with `rooms_df.to_csv('rooms.csv')`

The real building data gives you endless possibilities for analysis!